In [ ]:
! pip install -q kaggle

In [ ]:
! mkdir ~/.kaggle 

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d ruturajkanake/real-and-fake-news

 83% 34.0M/41.0M [00:00<00:00, 125MB/s]
100% 41.0M/41.0M [00:00<00:00, 163MB/s]


In [ ]:
! unzip real-and-fake-news.zip

Archive:  real-and-fake-news.zip
  inflating: Fake.csv                
  inflating: True.csv                


In [ ]:
import numpy as np
import pandas as pd
import re
import string

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

In [ ]:
fake['label']= 0
true['label'] = 1

In [ ]:
fake.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [ ]:
true.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [ ]:
dataset = pd.concat([fake, true])

In [ ]:
dataset.drop_duplicates(inplace=True)

In [ ]:
dataset = dataset.drop(['date'], axis=1)

In [ ]:
dataset.head()

,title,text,subject,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,0


In [ ]:
dataset['subject'].value_counts()

politicsNews       11220
worldnews           9991
News                9050
politics            6838
left-news           4459
Government News     1570
US_News              783
Middle-east          778
Name: subject, dtype: int64

In [ ]:
subject_mapping = {}

for i in range(0, len(dataset['subject'].unique())):
  subject_mapping[dataset['subject'].unique()[i]] = i+1

dataset['subject'] = dataset['subject'].map(subject_mapping)

In [ ]:
dataset['text'] = dataset['text'].str.lower()

In [ ]:
dataset.head()

,title,text,subject,label
0,Donald Trump Sends Out Embarrassing New Year’...,donald trump just couldn t wish all americans ...,1,0
1,Drunk Bragging Trump Staffer Started Russian ...,house intelligence committee chairman devin nu...,1,0
2,Sheriff David Clarke Becomes An Internet Joke...,"on friday, it was revealed that former milwauk...",1,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"on christmas day, donald trump announced that ...",1,0
4,Pope Francis Just Called Out Donald Trump Dur...,pope francis used his annual christmas day mes...,1,0


In [ ]:
def clean_text(text):
    
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('Reuters','',text)
    return text

dataset['text'] = dataset['text'].apply(lambda x:clean_text(x))

In [ ]:
stop = stopwords.words('english')
dataset['text'] = dataset['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
def lemmatize_words(text):
    wnl = nltk.stem.WordNetLemmatizer()
    lem = ' '.join([wnl.lemmatize(word) for word in text.split()])    
    return lem

dataset['text'] = dataset['text'].apply(lemmatize_words)

In [ ]:
y = dataset['label']
X_train, X_test, y_train, y_test = train_test_split(dataset['text'], y,test_size=0.4,random_state=42)

In [ ]:
count_vectorizer = CountVectorizer(stop_words='english')
count_train = count_vectorizer.fit_transform(X_train.values)
count_test = count_vectorizer.transform(X_test.values)
count_train.shape

(26813, 148359)

In [ ]:
clf = SVC(kernel='linear')

clf.fit(count_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
y_predict = clf.predict(count_test)

In [ ]:
from sklearn.metrics import accuracy_score

score = accuracy_score(y_test, y_predict)

print('Accuracy: ', score)

Accuracy:  0.9953569031103155
